In [30]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [42]:
head = cv2.imread('./data/0342/IMG_0342_1.tif', cv2.IMREAD_GRAYSCALE)
img = np.zeros((head.shape[0], head.shape[1], 4), dtype='uint8')

img[:,:,0] = head
img[:,:,1] = cv2.imread('./data/0342/IMG_0342_2.tif', cv2.IMREAD_GRAYSCALE)
img[:,:,2] = cv2.imread('./data/0342/IMG_0342_3.tif', cv2.IMREAD_GRAYSCALE)
img[:,:,3] = cv2.imread('./data/0342/IMG_0342_4.tif', cv2.IMREAD_GRAYSCALE)

cv2.imshow("Registered image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [43]:
# Brute-Force Matching with SIFT Descriptors

def SIFT(img):
    siftDetector= cv2.xfeatures2d.SIFT_create()
    kp, des = siftDetector.detectAndCompute(img, None)
    return kp, des

def matcher(des1, des2): 
    # create Matcher object
    bf_matcher = cv2.BFMatcher()

    # Match descriptors.
    matches = bf_matcher.match(des1, des2, None)  #Creates a list of all matches, just like keypoints

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x: x.distance)
    return matches

def getHomography(matches, kp1, kp2):
    points1 = np.zeros((len(matches), 2), dtype=np.float32)  #Prints empty array of size equal to (matches, 2)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = kp1[match.queryIdx].pt    #gives index of the descriptor in the list of query descriptors
        points2[i, :] = kp2[match.trainIdx].pt    #gives index of the descriptor in the list of train descriptors

    h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)
    return h

In [44]:
for i in [1,2,3]:
    # Find match points
    kp1, des1 = SIFT(img[:,:,0])
    kp2, des2 = SIFT(img[:,:,i])
    matches = matcher(des1, des2)

    # Use homography
    h = getHomography(matches, kp1, kp2)
    height, width, channels = img.shape
    img[:,:,i] = cv2.warpPerspective(img[:,:,i], h, (width, height))  #Applies a perspective transformation to an image.

    print("Estimated homography : \n",  h)

cv2.imshow("Registered image", img[:,:,:3])
cv2.waitKey(0)
cv2.destroyAllWindows()

Estimated homography : 
 [[ 1.00468876e+00 -4.63354652e-03 -7.42156692e+00]
 [ 4.66936122e-03  1.00267770e+00  2.53087947e+01]
 [ 2.10053446e-06 -1.14765900e-06  1.00000000e+00]]
Estimated homography : 
 [[ 1.00608470e+00 -3.60006229e-03  1.08584259e+01]
 [ 5.92347431e-03  1.00419981e+00  2.08415638e+01]
 [ 3.41684151e-06  1.17436066e-06  1.00000000e+00]]
Estimated homography : 
 [[-2.89629100e-01  2.89875718e-02  2.56992303e+02]
 [-7.46300488e-01 -5.16052248e-01  8.14692651e+02]
 [-8.73782596e-04 -5.82048430e-04  1.00000000e+00]]


In [45]:
cv2.imshow("Registered image", img[:,:,1:4])
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# Brute-Force Matching with SIFT Descriptors

def SIFT(img):
    siftDetector= cv2.xfeatures2d.SIFT_create()
    kp, des = siftDetector.detectAndCompute(img, None)
    return kp, des

def matcher(des1, des2): 
    # create Matcher object
    bf_matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)

    # Match descriptors.
    matches = bf_matcher.match(des1, des2, None)  #Creates a list of all matches, just like keypoints

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x: x.distance)


# Draw first 10 matches.
img3 = cv2.drawMatches(im1,kp1, im2, kp2, matches[:10], None)

cv2.imshow("Matches image", img3)
cv2.waitKey(0)

#Now let us use these key points to register two images. 
#Can be used for distortion correction or alignment
#For this task we will use homography. 
# https://docs.opencv.org/3.4.1/d9/dab/tutorial_homography.html

# Extract location of good matches.
# For this we will use RANSAC.
#RANSAC is abbreviation of RANdom SAmple Consensus, 
#in summary it can be considered as outlier rejection method for keypoints.
#http://eric-yuan.me/ransac/
#RANSAC needs all key points indexed, first set indexed to queryIdx
#Second set to #trainIdx. 

points1 = np.zeros((len(matches), 2), dtype=np.float32)  #Prints empty array of size equal to (matches, 2)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
   points1[i, :] = kp1[match.queryIdx].pt    #gives index of the descriptor in the list of query descriptors
   points2[i, :] = kp2[match.trainIdx].pt    #gives index of the descriptor in the list of train descriptors

#Now we have all good keypoints so we are ready for homography.   
# Find homography
#https://en.wikipedia.org/wiki/Homography_(computer_vision)
  
h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
height, width, channels = im2.shape
im1Reg = cv2.warpPerspective(im1, h, (width, height))  #Applies a perspective transformation to an image.
   
print("Estimated homography : \n",  h)

cv2.imshow("Registered image", im1Reg)
cv2.waitKey()